In [ ]:
!pip install requests beautifulsoup4 pandas

In [ ]:
pip install zenrows

In [9]:
pip install apscheduler


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 1.2 MB/s eta 0:00:00


In [10]:
from zenrows import ZenRowsClient
import requests

url = 'https://www.meti.go.jp/policy/external_economy/trade_control/wnlist.html'
apikey = 'cafbaf0b42acee3ab891441c64c48cb70d58605c'
params = {
  'url': url,
  'apikey': apikey,
	'js_render': 'true',
	'json_response': 'true',
}
response = requests.get('https://api.zenrows.com/v1/', params=params)
print(response.text)

{"html":"<html xmlns=\"http://www.w3.org/1999/xhtml\" xml:lang=\"ja\" lang=\"ja\"><head>\n<!-- globalNaviSelect:政策について -->\n<!-- sideNaviSelect:政策一覧 -->\n<!-- sideNaviSubSelect:対外経済 -->\n<meta http-equiv=\"Content-Type\" content=\"text/html; charset=utf-8\">\n<meta http-equiv=\"Content-Style-Type\" content=\"text/css\">\n<meta http-equiv=\"Content-Script-Type\" content=\"text/javascript\">\n<meta http-equiv=\"X-UA-Compatible\" content=\"IE=100\">\n<title>新着情報一覧（METI/経済産業省）</title>\n<meta name=\"viewport\" content=\"width=device-width,initial-scale=1.0,minimum-scale=1.0,user-scalable=yes\">\n<meta name=\"format-detection\" content=\"telephone=no\">\n<meta name=\"keywords\" content=\"貿易,申請,手続き,法令,様式,貿易管理,経済産業省,METI,輸入,輸出,関税割当,電子申請\">\n<meta name=\"description\" content=\"\">\n<link href=\"/favicon.ico\" rel=\"shortcut icon\">\n<link href=\"/jquery/jquery-ui.css\" rel=\"stylesheet\" type=\"text/css\">\n<link href=\"/css/base2014.css\" rel=\"stylesheet\" type=\"text/css\" media=\"all\">\n<

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from apscheduler.schedulers.blocking import BlockingScheduler
import os

def crawl_japan_customs():
    url = 'https://www.meti.go.jp/policy/external_economy/trade_control/wnlist.html'
    apikey = 'cafbaf0b42acee3ab891441c64c48cb70d58605c'
    params = {
        'url': url,
        'apikey': apikey,
        'js_render': 'true',
        'json_response': 'false',
    }

    req = requests.get('https://api.zenrows.com/v1/', params=params)

    if req.status_code == requests.codes.ok:
        print("접속 성공")
        html = BeautifulSoup(req.text, "html.parser")

        dates, titles, links, categories, attachments = [], [], [], [], []

        notices = html.select("table tbody tr")
        for notice in notices:
            date_tag = notice.select_one("td.csv2table-table-td")
            date = date_tag.get_text(strip=True) if date_tag else None
            dates.append(date)

            title_tag = notice.select_one("td:nth-child(2) a")
            if title_tag:
                title = title_tag.get_text(strip=True)
                link = title_tag['href']
            else:
                title = None
                link = None
            titles.append(title)
            links.append(link)

            category_tag = notice.select_one("td:nth-child(3) img")
            category = category_tag['alt'] if category_tag else None
            categories.append(category)

            attachment_tag = notice.select_one("td:nth-child(4) span")
            attachment = attachment_tag.get_text(strip=True) if attachment_tag else None
            attachments.append(attachment)

        # 새로 크롤링한 데이터프레임 생성
        new_df = pd.DataFrame({
            '날짜': dates,
            '제목': titles,
            '링크': links,
            '분류': categories,
            '첨부파일': attachments
        })

        # 기존 CSV 파일이 있으면 읽기
        file_name = 'jpcustoms_notices.csv'
        if os.path.exists(file_name):
            existing_df = pd.read_csv(file_name)
            # 중복된 제목(또는 날짜)을 기준으로 데이터프레임 합치기
            combined_df = pd.concat([existing_df, new_df]).drop_duplicates(subset=['제목'], keep='last')
        else:
            combined_df = new_df  # 파일이 없으면 새 데이터프레임 사용

        # 데이터프레임을 CSV 파일로 저장
        combined_df.to_csv(file_name, index=False, encoding='utf-8-sig')
        print("데이터가 jpcustoms_notices.csv 파일에 저장되었습니다.")
    else:
        print("접속 실패")

scheduler = BlockingScheduler()
scheduler.add_job(crawl_japan_customs, 'interval', hours=24)  # 24시간마다 실행
print("자동화된 크롤링 시작")
scheduler.start()


자동화된 크롤링 시작
